In [1]:
!pip install pandas sentence-transformers chromadb openai faker


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 15.6 MB/s eta 0:00:0

# ChromaDB + SentenceTransformer Setup

## Imports
```python
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import random
from faker import Faker


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import random
from faker import Faker

persist_directory = 'db'
client = chromadb.Client(Settings(persist_directory=persist_directory))

model = SentenceTransformer('all-MiniLM-L6-v2')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
fake = Faker()
Faker.seed(42)

def generate_transaction():
    return {
        "transaction_id": fake.uuid4(),
        "user_id": random.randint(1000, 9999),
        "amount": round(random.uniform(10, 10000), 2),
        "merchant": random.choice(["Amazon", "Flipkart", "Swiggy", "Paytm", "IRCTC", "MakeMyTrip", "UnknownMerchant"]),
        "timestamp": fake.date_time_between(start_date="-30d", end_date="now"),
        "location": fake.city(),
        "device_id": fake.mac_address()
    }

df = pd.DataFrame([generate_transaction() for _ in range(10000)])

df["text"] = df.apply(lambda row: f"User {row['user_id']} spent ₹{row['amount']} at {row['merchant']} on {row['timestamp']} from {row['location']} using device {row['device_id']}", axis=1)

embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)

collection_name = "fraud_detection_transactions"

try:
    collection = client.get_collection(name=collection_name)
except:
    collection = client.create_collection(name=collection_name)

for i, embedding in enumerate(embeddings):
    metadata = df.iloc[i].to_dict()
    metadata['timestamp'] = str(metadata['timestamp'])
    collection.add(
        documents=[df["text"].iloc[i]],
        metadatas=[metadata],
        ids=[str(df["transaction_id"].iloc[i])],
        embeddings=[embedding]
    )

print(f"✅ Data successfully stored in ChromaDB! Total {len(df)} transactions.")

def search_similar_transactions(query_text, top_k=5):
    query_embedding = model.encode([query_text])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )
    return results

query = "User spent ₹10000 at night from new device at Amazon"
results = search_similar_transactions(query)

for i, result in enumerate(results["documents"], 1):
    print(f"\n--- Match #{i} ---")
    print(f"Transaction ID: {results['ids'][i-1]}")
    print(f"Text: {result}")
    print(f"Metadata: {results['metadatas'][i-1]}")


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

✅ Data successfully stored in ChromaDB! Total 10000 transactions.

--- Match #1 ---
Transaction ID: ['71d7c324-a5a5-4349-816b-053154d77fbe', 'c91c253b-2821-4305-b224-23ef4a7f7de7', '352408c0-593a-4fbb-893c-f12e71f379a9', 'bd59b23d-7953-41a8-96ae-6406a0768006', '952a47a8-9d12-481e-bb0d-5c9257dd3f8c']
Text: ['User 3704 spent ₹8968.19 at Amazon on 2025-03-20 02:29:26.108410 from New Ericland using device 72:3c:0f:62:2d:de', 'User 6333 spent ₹4080.17 at Amazon on 2025-03-20 07:40:32.045233 from Davidshire using device 92:51:dd:79:d6:03', 'User 3320 spent ₹1506.18 at Amazon on 2025-03-24 14:02:26.033960 from West Michaelburgh using device 8a:2d:b7:14:2e:96', 'User 8412 spent ₹9103.67 at Amazon on 2025-03-23 04:23:15.600921 from New Jamesstad using device 56:04:cf:e3:b3:48', 'User 3915 spent ₹1131.47 at Amazon on 2025-04-03 10:55:26.567207 from East Gina using device 86:1f:d4:93:e6:14']
Metadata: [{'amount': 8968.19, 'device_id': '72:3c:0f:62:2d:de', 'text': 'User 3704 spent ₹8968.19 at Amaz

In [5]:
fake = Faker()
Faker.seed(42)

def generate_transaction():
    return {
        "transaction_id": fake.uuid4(),
        "user_id": random.randint(1000, 9999),
        "amount": round(random.uniform(10, 10000), 2),
        "merchant": random.choice(["Amazon", "Flipkart", "Swiggy", "Paytm", "IRCTC", "MakeMyTrip", "UnknownMerchant"]),
        "timestamp": fake.date_time_between(start_date="-30d", end_date="now"),
        "location": fake.city(),
        "device_id": fake.mac_address()
    }

df = pd.DataFrame([generate_transaction() for _ in range(10000)])

df["text"] = df.apply(lambda row: f"User {row['user_id']} spent ₹{row['amount']} at {row['merchant']} on {row['timestamp']} from {row['location']} using device {row['device_id']}", axis=1)

embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)

collection_name = "fraud_detection_transactions"

try:
    collection = client.get_collection(name=collection_name)
except:
    collection = client.create_collection(name=collection_name)

for i, embedding in enumerate(embeddings):
    metadata = df.iloc[i].to_dict()
    metadata['timestamp'] = str(metadata['timestamp'])
    collection.add(
        documents=[df["text"].iloc[i]],
        metadatas=[metadata],
        ids=[str(df["transaction_id"].iloc[i])],
        embeddings=[embedding]
    )

print(f"✅ Data successfully stored in ChromaDB! Total {len(df)} transactions.")

def search_similar_transactions(query_text, top_k=5):
    query_embedding = model.encode([query_text])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k
    )
    return results

query = "User spent ₹10000 at night from new device at Amazon"
results = search_similar_transactions(query)

for i, result in enumerate(results["documents"], 1):
    print(f"\n--- Match #{i} ---")
    print(f"Transaction ID: {results['ids'][i-1]}")
    print(f"Text: {result}")
    print(f"Metadata: {results['metadatas'][i-1]}")



--- Query: Large transaction at unusual time ---

--- Match #1 ---
Transaction ID: ['b5fb3028-cd0a-495c-8615-730397aa441c', '24a81aa9-8fab-4db8-801f-2c03292c2c78', '014883a1-0f71-4d0a-8be9-c8296812ebb9', 'c0b739f3-b0c4-4b27-83cd-578a8e89bf1c', '91bd7c83-0d9f-48f0-a0b4-f5a26b3c8bbd']
Text: ['User 4145 spent ₹9222.17 at Amazon on 2025-04-06 14:11:49.045014 from Karenville using device 4a:f4:d7:c5:08:c5', 'User 6785 spent ₹6199.17 at Amazon on 2025-03-28 19:13:17.671089 from Port Karen using device 50:b2:a2:4c:4f:0b', 'User 6240 spent ₹5623.37 at Amazon on 2025-03-22 02:28:30.921229 from Lake Cheyennechester using device b4:8c:c7:b9:67:23', 'User 4073 spent ₹8849.68 at Amazon on 2025-03-22 21:03:28.920920 from Port Theresa using device a8:01:db:a8:d3:47', 'User 8140 spent ₹9265.15 at Amazon on 2025-04-06 12:05:27.547670 from Port Justin using device c8:05:4d:77:08:36']


TypeError: list indices must be integers or slices, not str

In [6]:
def search_similar_transactions(query_text, top_k=5):
    query_embedding = model.encode([query_text])
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        where_document={"$contains": "Amazon"},
        where={"amount": {"$gt": 5000}}
    )
    return results

queries = [
    "Large transaction at unusual time",
    "High-value purchase from a new device",
    "Transaction from an unfamiliar location",
    "Multiple small transactions in quick succession from the same user",
    "Transaction significantly deviating from user's spending pattern",
    "Transaction to an unknown merchant with high value"
]

for query in queries:
    print(f"\n--- Query: {query} ---")
    results = search_similar_transactions(query)

    for i, result in enumerate(results["documents"], 1):
        print(f"\n--- Match #{i} ---")
        print(f"Transaction ID: {results['ids'][i-1]}")
        print(f"Text: {result}")
        metadata = results['metadatas'][i-1][0]
        print(f"Amount: ₹{metadata['amount']}")
        print(f"Merchant: {metadata['merchant']}")
        print(f"Location: {metadata['location']}")
        print(f"Timestamp: {metadata['timestamp']}")
        print(f"Device ID: {metadata['device_id']}")



--- Query: Large transaction at unusual time ---

--- Match #1 ---
Transaction ID: ['b5fb3028-cd0a-495c-8615-730397aa441c', '24a81aa9-8fab-4db8-801f-2c03292c2c78', '014883a1-0f71-4d0a-8be9-c8296812ebb9', 'c0b739f3-b0c4-4b27-83cd-578a8e89bf1c', '91bd7c83-0d9f-48f0-a0b4-f5a26b3c8bbd']
Text: ['User 4145 spent ₹9222.17 at Amazon on 2025-04-06 14:11:49.045014 from Karenville using device 4a:f4:d7:c5:08:c5', 'User 6785 spent ₹6199.17 at Amazon on 2025-03-28 19:13:17.671089 from Port Karen using device 50:b2:a2:4c:4f:0b', 'User 6240 spent ₹5623.37 at Amazon on 2025-03-22 02:28:30.921229 from Lake Cheyennechester using device b4:8c:c7:b9:67:23', 'User 4073 spent ₹8849.68 at Amazon on 2025-03-22 21:03:28.920920 from Port Theresa using device a8:01:db:a8:d3:47', 'User 8140 spent ₹9265.15 at Amazon on 2025-04-06 12:05:27.547670 from Port Justin using device c8:05:4d:77:08:36']
Amount: ₹9222.17
Merchant: Amazon
Location: Karenville
Timestamp: 2025-04-06 14:11:49.045014
Device ID: 4a:f4:d7:c5:08:c

In [11]:
def add_more_data_for_user(user_id, num_transactions=100):
    new_transactions = []
    for _ in range(num_transactions):
        transaction = generate_transaction()
        transaction["user_id"] = user_id
        new_transactions.append(transaction)

    new_df = pd.DataFrame(new_transactions)
    new_df["text"] = new_df.apply(lambda row: f"User {row['user_id']} spent ₹{row['amount']} at {row['merchant']} on {row['timestamp']} from {row['location']} using device {row['device_id']}", axis=1)

    new_embeddings = model.encode(new_df["text"].tolist(), show_progress_bar=True)

    for i, embedding in enumerate(new_embeddings):
        metadata = new_df.iloc[i].to_dict()
        metadata['timestamp'] = str(metadata['timestamp'])
        collection.add(
            documents=[new_df["text"].iloc[i]],
            metadatas=[metadata],
            ids=[str(new_df["transaction_id"].iloc[i])],
            embeddings=[embedding]
        )

    print(f"Added {num_transactions} transactions for user {user_id}")

add_more_data_for_user(1001, 500)


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Added 500 transactions for user 1001


In [16]:
def get_user_transactions(user_id):
    results = collection.query(
        #where={"user_id": str(user_id)}, # Query using the user_id
        #n_results=1000, # Adjust as needed, or use all() for all results
        # Include either query_embeddings, query_texts, query_images, or query_uris:
        query_texts=[f"User {user_id}"],  # Example: Search for documents containing the user ID in the text
        n_results=1000  # Adjust as needed, or use all() for all results
    )

    # Display top matches with detailed metadata
    for i, result in enumerate(results["documents"], 1):
        print(f"\n--- Match #{i} ---")
        print(f"Transaction ID: {results['ids'][i-1]}")
        print(f"Text: {result}")
        # Access metadata as a dictionary from the list of metadata
        metadata = results['metadatas'][i-1][0] # Access the first element which is the dictionary
        print(f"Amount: ₹{metadata['amount']}") # Highlight the amount
        print(f"Merchant: {metadata['merchant']}") # Highlight the merchant
        print(f"Location: {metadata['location']}") # Highlight the location
        print(f"Timestamp: {metadata['timestamp']}")  # Highlight the timestamp
        print(f"Device ID: {metadata['device_id']}") # Highlight the device ID

### SEMANTIC SEARCH OF ALL TRANSACTION DONE BY A SPECIFIC USER USING SIMILARITY SEARCH IN CHROMADB


In [17]:
get_user_transactions(1001)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.9MiB/s]



--- Match #1 ---
Transaction ID: ['07ca9e66-3588-4500-865a-85b5bd4d3eef', '7b46bbd9-7426-4b55-912c-06715abd6912', 'ce2d422e-ca73-41a8-af9f-35131eaae3cf', '7a8a29bb-9603-459a-8bfe-20a4ac184b2b', 'cfbd6260-8ebb-4eef-aa2f-b59d1a26394e', '536ecf1f-70c9-47da-b65b-c29333334e4d', 'e33cd384-5ac7-4bf6-aadb-576687d8c006', '0360bd53-3da1-4147-93b5-cc62fd924451', 'a189635a-ba98-4249-84a7-d56e9376711a', '738583bc-60c3-4921-9064-72017ac4c286', '09ba054c-e2a4-4ab3-a195-57b3b6f3b4ad', 'f3abd246-7570-4730-a5e7-9add68c87ed8', '2143ad90-2243-420a-addd-c3e6329cb94e', 'fa1b1be5-9635-4293-ac77-a882e3b0f7be', '50bd724d-7b5b-4a48-8683-8bed11bf06d3', '792bdc45-0319-4f2b-bc6f-69ed690417f4', 'db99d834-c606-4146-876e-f11240b8357b', 'dbdfdb5b-5c89-45fd-9edf-c8b19118da98', '9a2dcb37-866e-4895-85ef-dbec9d6b0276', '7d7e604b-c0ed-407b-b701-8b7069967499', 'e462f756-0b28-4cb3-965f-71bda2b2c067', '4feb6670-57a6-4200-9177-82ab613069c7', '12510712-50d8-4aca-8744-3f73d33981ff', '536836c7-3ba2-4bff-afd6-952ab3a63585', '8acf

In [ ]:
#CREW AI FRAMEWORK (IN PROGRESS)

In [ ]:
import crewai